# This notebook contains code to run CellPose for cell segmentation

## Adding last modified data

In [2]:
#Adding last modified date
from datetime import datetime
from IPython.display import display, Markdown
todays_date = str(datetime.now().date())
time = str(str(datetime.now().time()))
author = "Keerthana Arun"
display(Markdown(f'Last modified on {todays_date} at {time} by {author}'))

Last modified on 2024-04-19 at 02:17:11.944995 by Keerthana Arun

## Installing and importing package

### Installation

In [3]:
# # %pip install "opencv-python-headless<4.3"
# # %pip install cellpose==2.0
# # %pip install pandas
# %pip install scikit-image
# # %pip install multipagetiff

### Importing packages

In [4]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import cellpose
%matplotlib inline
# mpl.rcParams['figure.dpi'] = 300
import csv
from urllib.parse import urlparse
from cellpose import models, core, utils, io
import torch
from os import listdir
from os.path import isfile, join
import multipagetiff as mtif
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from cellpose import plot
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
%pip list

Package                   Version
------------------------- --------------
anndata                   0.10.3
anyio                     4.3.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
array-api-compat          1.4
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.2.0
Babel                     2.14.0
beautifulsoup4            4.12.3
bleach                    6.1.0
cachetools                5.3.3
cellpose                  2.0
certifi                   2024.2.2
cffi                      1.16.0
charset-normalizer        3.3.2
comm                      0.2.2
contourpy                 1.2.1
cycler                    0.12.1
debugpy                   1.8.1
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.0.1
fastjsonschema            2.19.1
fastremap                 1.14.1
filelock                  3.13.4
fonttools                 4.51.0
fqdn              

## Processing

List of images to be analysed

In [6]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

Given a list of files with parameters, run CellPoseModel

In [7]:
def runCellPoseModel(filePath, condition, diameterParam, flowThresholdParam, cellprobThresholdParam, outputPath):
    files = [f for f in listdir(filePath) if isfile(join(filePath, f))]
    spec = "zoverlay"
    files = [f for f in files if spec in f]
    files = [f for f in files if condition in f]
    files = sorted(files)
    nimg = len(files)
    print(nimg)
    os.chdir(filePath)
    imgs = [skimage.io.imread(f) for f in files]
    channelsParam = [[0,3]]
    modelTypeParam = "cyto2"
 
    use_GPU = core.use_gpu()
    model = models.CellposeModel(gpu=use_GPU, model_type=modelTypeParam)
    masks, flows, styles = model.eval(imgs, diameter=int(diameterParam), flow_threshold=float(flowThresholdParam), channels=channelsParam, cellprob_threshold=int(cellprobThresholdParam))
    outputImagesPath = os.path.join(outputPath, "Images")
    outputSegPath = os.path.join(outputPath, "Segmentation")
    if not os.path.exists(outputImagesPath):
        os.makedirs(outputImagesPath) 
    if not os.path.exists(outputSegPath):
        os.makedirs(outputSegPath) 
    os.chdir(outputSegPath)
    io.masks_flows_to_seg(imgs, masks, flows, diams=int(diameterParam), file_names=files, channels=channelsParam)
    os.chdir(outputImagesPath)
    io.save_to_png(imgs, masks, flows, files)
    return




Running CellPose with human-trained model

In [8]:
def runCellPoseModelHuman(filePath, condition, diameterParam, flowThresholdParam, cellprobThresholdParam, outputPath):
    files = [f for f in listdir(filePath) if isfile(join(filePath, f))]
    spec = "zoverlay"
    files = [f for f in files if spec in f]
    files = [f for f in files if condition in f]
    files = sorted(files)
    nimg = len(files)
    print(nimg)
    os.chdir(filePath)
    imgs = [skimage.io.imread(f) for f in files]
    channelsParam = [[0,3]]
    modelTypeParam = "cyto2"
    pretrainedModel = "/home/mzo5929/Keerthana/MoiseEtAl/Model/Figure6New-5"
    use_GPU = core.use_gpu()
    model = models.CellposeModel(gpu=use_GPU, model_type=modelTypeParam, pretrained_model= pretrainedModel)
    masks, flows, styles = model.eval(imgs, diameter=int(diameterParam), flow_threshold=float(flowThresholdParam), channels=channelsParam, cellprob_threshold=int(cellprobThresholdParam))
    outputImagesPath = os.path.join(outputPath, "Images")
    outputSegPath = os.path.join(outputPath, "Segmentation")
    if not os.path.exists(outputImagesPath):
        os.makedirs(outputImagesPath) 
    if not os.path.exists(outputSegPath):
        os.makedirs(outputSegPath) 
    os.chdir(outputSegPath)
    io.masks_flows_to_seg(imgs, masks, flows, diams=int(diameterParam), file_names=files, channels=channelsParam)
    os.chdir(outputImagesPath)
    io.save_to_png(imgs, masks, flows, files)
    return


Function to run tasks in parallel

In [9]:
import csv
from concurrent.futures import ProcessPoolExecutor

def runTaskFromCsv(row):
    # Unpack the row directly into the function call
    runCellPoseModelHuman(*row)


## Main 

### Input parameters
Input path to raw images, csv files for parameters to input for CellPose

In [15]:
# dataPath = ""
csvParameters = "/home/mzo5929/Keerthana/MoiseEtAl/rawData/parametersf6.csv"
# outputPath = ""

In [16]:
parameterDf = pd.read_csv(csvParameters)
parameterDf.shape

(6, 6)

### Main Function

In [17]:
tasks = []
index = 0
with open(csvParameters, 'r') as file:
    os.chdir("/home/mzo5929/Keerthana/MoiseEtAl/extractionScripts")
    reader = csv.reader(file)
    next(reader)  # Skip the header row if there is one
    for row in reader:
        if(index <5):
            index = index+1
            continue
        tasks.append(row)  # Collect tasks
        print(row)
        runTaskFromCsv(row)

['/home/mzo5929/Keerthana/MoiseEtAl/rawData/figure6/2024_03_08_acetyltub_kocells/SM9 HDAC6KO Flow', 'flow', '180', '0.4', '0', '/home/mzo5929/Keerthana/MoiseEtAl/extractedData/segmentationResults/figure6/2024_03_08_acetyltub_kocells/SM9 HDAC6KO Flow']
47
